In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.lines as mlines
from scipy.optimize import minimize
from matplotlib import pyplot as plt
from joblib import Parallel, delayed
from sklearn.decomposition import NMF
from matplotlib.ticker import ScalarFormatter
np.random.seed(3000000)

In [ ]:
user_artists = pd.read_csv('user_artists.dat', sep='\t')

artist_counts = user_artists['artistID'].value_counts()
artists_filtered = artist_counts[artist_counts >= 30].index
user_artists_filtered = user_artists[user_artists['artistID'].isin(artists_filtered)]

user_counts = user_artists_filtered['userID'].value_counts()
users_filtered = user_counts[user_counts >= 30].index
user_artists_final = user_artists_filtered[user_artists_filtered['userID'].isin(users_filtered)]

rating_matrix = pd.pivot_table(user_artists_final, index = 'userID', columns = 'artistID', aggfunc = lambda x: 1, fill_value = 0)

In [ ]:
d_ = 3
d = d_ ** 2

model = NMF(n_components = d_, init = 'nndsvda', max_iter = 10000)
W = model.fit_transform(rating_matrix)
H = model.components_

In [ ]:
def generate_data(W, H, d):
    
    phi_true = np.zeros((W.shape[0], H.shape[1], d))
    psi_true = np.zeros((W.shape[0], H.shape[1], d))
            
    theta_true = np.eye(int(np.sqrt(d))).ravel()
    theta_norm = np.linalg.norm(theta_true)
    theta_true = theta_true / theta_norm
    
    for i in range(W.shape[0]):
        
        for j in range(H.shape[1]):
            
            temp = np.outer(W[i], H[:, j])
            phi_true[i, j] = temp.ravel()
            psi_true[i, j] = phi_true[i, j] + 1e-3 * np.random.normal(0, 1, d)
            
    max_value = np.max(np.dot(phi_true, theta_true))
    phi_true = phi_true / max_value
    psi_true = psi_true / max_value
            
    return phi_true, psi_true, theta_true

In [ ]:
phi_true, psi_true, theta_true = generate_data(W, H, d)
phi_true.shape, psi_true.shape, theta_true

In [ ]:
# calculate the beta
def get_beta(rho, delta, V_bar, theta_true):
    
    ld = 0.05
    beta = rho * np.sqrt(2 * np.log((np.sqrt(np.linalg.det(V_bar)) * (np.linalg.det(ld * np.eye(d)) ** (-1/2))) / delta)) + np.sqrt(ld) * np.linalg.norm(theta_true)
    
    return beta

In [ ]:
# slsqp
def get_decision(psi_action, theta_hat, V_bar, beta, d):
    
    def maximize_reward(theta, psi_a):

        return -1.0 * psi_a.dot(theta)
    
    # constraint confidence set: || theta_hat - theta ||_V_bar < Beta
    def constraint(theta, theta_hat, V_bar, beta):
        
        temp = np.array(theta_hat - theta.reshape(-1, 1))
        norm = np.sqrt(temp.reshape(-1, 1).T.dot(V_bar).dot(temp.reshape(-1, 1)))
        
        return beta - norm[0][0]
    
    # constraints for optimization
    beta_constraint = {'type': 'ineq', 'fun' : lambda theta: constraint(theta, theta_hat, V_bar, beta)}
    
    theta_list = []
    reward_list = []
    
    for psi_a in psi_action:
        
        res = minimize(maximize_reward, x0 = np.ones(d), args = (psi_a), method = 'SLSQP', constraints = [beta_constraint], options = {'ftol': 1e-3, 'eps': 1e-10, 'maxiter': 1e6, 'disp': False})
        theta_list.append(res.x)
        reward_list.append(psi_a.dot(res.x))
        
    decision = np.argmax(reward_list)
    theta_tilde = theta_list[decision]
    
    return decision, theta_tilde

In [ ]:
def prepare_plot_data(data_list, trials, iterations, num_agent, every_num_point):
    
    new_list = []

    for T in range(trials):

        new_list.append([item[1] for item in data_list if item[0] == T])

    new_list = np.array(new_list).reshape(trials, iterations).tolist()
    
    data_value = np.zeros(iterations)
    
    for T in range(trials):
        
        data_value += np.array(new_list[T])
        
    data_value = data_value / trials
    
    x_value = [0]
    y_value = [data_value[0] / num_agent[0]]
    
    for num in range(int(iterations / every_num_point)):
        
        x_value.append((num + 1) * every_num_point - 1)
        y_value.append(data_value[(num + 1) * every_num_point - 1] / num_agent[0])
    
    return x_value, y_value

In [ ]:
R = 0.05
ld = 0.05
trials = 1
alpha = 0.7
num_agent = [1]
baseline_idx = 5
delta_value = 1e-2
iterations = 10000000
every_num_point = 25
optimal_alg = []
optimal_ECC = []
optimal_sw = []
optimal_sw_UCB = []
reward_alg = []
reward_ECC = []
reward_sw = []
reward_sw_UCB = []
baseline_alg = []
baseline_ECC = []
baseline_sw = []
baseline_sw_UCB = []
cummulative_regret_alg = []
cumulative_violate_alg = []
cumulative_baseline_alg = []
cummulative_regret_ECC = []
cumulative_violate_ECC = []
cumulative_baseline_ECC = []
cummulative_regret_sw = []
cumulative_violate_sw = []
cumulative_baseline_sw = []
cummulative_regret_sw_UCB = []
cumulative_violate_sw_UCB = []
cumulative_baseline_sw_UCB = []
L = np.max(np.linalg.norm(phi_true, axis = 2))

In [ ]:
phi_true_1 = phi_true.copy()
phi_true_1[:, :, -1] = 0

phi_true_2 = phi_true.copy()
phi_true_2[:, :, 4] = 0

phi_true_3 = phi_true.copy()
phi_true_3[:, :, 0] = 0

phi_true_group = np.array([phi_true, phi_true_1, phi_true_2, phi_true_3])

In [ ]:
psi_true_1 = psi_true.copy()
psi_true_1[:, :, -1] = 0

psi_true_2 = psi_true.copy()
psi_true_2[:, :, 4] = 0

psi_true_3 = psi_true.copy()
psi_true_3[:, :, 0] = 0

psi_true_group = np.array([psi_true, psi_true_1, psi_true_2, psi_true_3])

In [ ]:
rewards_group = np.array([phi_true_group[i].dot(theta_true) for i in range(np.shape(phi_true_group)[0])])

In [ ]:
# generate the index set for context
sample_id = np.random.randint(0, np.shape(phi_true)[0], size=(trials, iterations))

# generate the noise of reward for each iteration
noise = np.random.normal(0, 1e-3, size = (num_agent[-1], trials, iterations))

# calculate r_l and r_h
result_list = np.dot(phi_true_group[0], theta_true)
sorted_list = np.sort(result_list, axis=1)
selected_list = sorted_list[:, -baseline_idx]
r_h = np.max(selected_list)
r_l = np.min(selected_list)
            
# gererate rho_bar
rho_bar = np.random.uniform(1e-10, alpha * r_l / (np.linalg.norm(theta_true) + r_h), size = (trials, iterations))

# generate zeta
zeta_data = np.random.normal(0, 1e-3, (trials, iterations, len(theta_true)))
zeta_zero = zeta_data - np.mean(zeta_data, axis = 2, keepdims = True)
zeta = zeta_zero / np.linalg.norm(zeta_zero, axis = 2, keepdims = True)

In [ ]:
# our setting
start = time.time()

for M in num_agent:
    
    for T in range(trials):
        
        total_regret = 0
        total_reward = 0
        total_violate = 0
        total_baseline = 0
        cummulative_regret = []
        cummulative_violate = []
        cummulative_baseline = []
        optimal_list = []
        reward_list = []
        baseline_list = []
        
        t_last = 0
        V_last = ld * np.eye(d)
        W_syn = np.zeros((d, d))
        U_syn = np.zeros((d, 1))
        
        W_new_list = [np.zeros((d, d)) for i in range(M)]
        U_new_list = [np.zeros((d, 1)) for i in range(M)]
        
        B = (iterations * np.log(M * iterations)) / (d * M)
        
        syn = 0

        for t in range(1, iterations + 1): 
            
            index = sample_id[T][t-1]
            
            for i in range(M):
                
                phi = phi_true_group[i][index]
                psi = psi_true_group[i][index]
                original = rewards_group[i][index]

                x_star = np.argmax(np.dot(np.array(psi), theta_true))
                optimal = original[x_star]
                
                x_b = np.argsort(original)[::-1][baseline_idx]
                r_b = original[x_b]
                
                V_bar = ld * np.eye(d) + W_syn + W_new_list[i]
                theta_hat = np.dot(np.linalg.inv(V_bar), (U_syn + U_new_list[i]))

                # construct the confidence ellipsoid beta
                beta = get_beta(rho = np.sqrt(1 + R ** 2), delta = delta_value / 2, V_bar = V_bar, theta_true = theta_true)
                
                #construct the trimmed action set
                tas = (psi.dot(theta_hat) >= beta * L / np.sqrt(np.min(np.linalg.eigvals(V_bar))) + (1 - alpha) * r_b)
                phi_set = phi[tas.ravel()]
                psi_set = psi[tas.ravel()]
                original_set = original[tas.ravel()]
                
                # get the best action
                if (psi_set.size != 0) and (np.min(np.linalg.eigvals(V_bar)) >= np.square(2 * L * beta / (alpha * r_b))):
                
                    decision, theta_tilde = get_decision(psi_set, theta_hat, V_bar, beta, d)
                    psi_new = psi_set[decision]
                    y = original_set[decision]
                    
                else:
                    
                    decision = x_b
                    total_baseline += 1
                    psi_new = (1 - rho_bar[T][t-1]) * psi[decision] + rho_bar[T][t-1] * zeta[T][t-1]
                    y = (1 - rho_bar[T][t-1]) * original[decision] + rho_bar[T][t-1] * np.dot(zeta[T][t-1], theta_true)
                    
                regret = optimal - y
                total_regret = total_regret + regret
                total_reward = total_reward + y

                # update W_new and U_new
                W_new_list[i] = W_new_list[i] + np.outer(psi_new, psi_new)
                U_new_list[i] = U_new_list[i] + psi_new.reshape(-1, 1) * (y + noise[i][T][t-1])
                V = ld * np.eye(d) + W_syn + W_new_list[i]
                
                LHS_condition = np.log(np.linalg.det(V) / np.linalg.det(V_last)) * (t - t_last)
                
                if LHS_condition >= B:
                    
                    syn = 1
                    
                if y < (1 - alpha) * r_b:
                    
                    total_violate += 1
                    
            if syn == 1:
                
                W_syn = W_syn + np.sum(W_new_list, axis=0)
                U_syn = U_syn + np.sum(U_new_list, axis=0)
                
                W_new_list = [np.zeros((d, d)) for i in range(M)]
                U_new_list = [np.zeros((d, 1)) for i in range(M)]
                t_last = t
                V_last = ld * np.eye(d) + W_syn
                
                syn = 0
                
            cummulative_regret.append(total_regret)
            cummulative_violate.append(total_violate)
            cummulative_baseline.append(total_baseline)
            optimal_list.append(np.max(np.dot(phi, theta_true)))
            reward_list.append(y)
            baseline_list.append((1 - alpha) * r_b)
            
        cummulative_regret_alg.append((T, cummulative_regret))
        cumulative_violate_alg.append((T, cummulative_violate))
        cumulative_baseline_alg.append((T, cummulative_baseline))
        optimal_alg.append((T, optimal_list))
        reward_alg.append((T, reward_list))
        baseline_alg.append((T, baseline_list))
        
end = time.time()
print('Finished! The total time we use is: ', end - start)

In [ ]:
x_value, y_alg = prepare_plot_data(cummulative_regret_alg, trials, iterations, num_agent, every_num_point)
x_value_r, y_optimal_alg = prepare_plot_data(optimal_alg, trials, iterations, num_agent, 1)
x_value_r, y_reward_alg = prepare_plot_data(reward_alg, trials, iterations, num_agent, 1)
x_value_r, y_baseline_alg = prepare_plot_data(baseline_alg, trials, iterations, num_agent, 1)

In [ ]:
# plot the figure
plt.figure(figsize=(12, 8), dpi=300)

colors = (['black', 'blue', 'darkgreen', 'purple', 'darkred', 'grey'])
markers = ['*', 's', 'o', 'X', '^', 'P']

plt.rcParams['xtick.labelsize'] = 25
plt.rcParams['ytick.labelsize'] = 25
plt.rc('legend', fontsize = 25)

ax = plt.gca()
ax.ticklabel_format(style='sci', axis='both', scilimits=(0, 0), useOffset=False)

plt.plot(x_value, y_alg, label = 'Algorithm', color = colors[1], linewidth=3)
plt.scatter(x_value[::50000], y_alg[::50000], label = 'Algorithm', marker = markers[1], color = colors[1], s=300)

legend_elements = [mlines.Line2D([0], [0], color=colors[1], lw = 5, label = 'DiSC-UCB', marker = markers[1], markersize = 15)]

plt.grid(True)
plt.xlabel('round,t', fontsize=25)
plt.ylabel('cumulative regret Rt', fontsize=25)
plt.title('movielens data', fontsize=25)
plt.legend(handles=legend_elements)
plt.show()
plt.close()

In [ ]:
# plot the figure
plt.figure(figsize=(12,8), dpi=300)

colors = (['black', 'blue', 'darkgreen', 'purple', 'darkred', 'grey'])
markers = ['*', 's', 'o', 'X', '^', 'P']

plt.rcParams['xtick.labelsize'] = 25
plt.rcParams['ytick.labelsize'] = 25
plt.rc('legend', fontsize = 25)

ax = plt.gca()
ax.ticklabel_format(style='sci', axis='both', scilimits=(0, 0), useOffset=False)

plt.plot(x_value_r[::1000], y_optimal_alg[::1000], label = 'Optimal-Reward', linestyle = '--', color = colors[0], linewidth=3)
plt.plot(x_value_r[::1000], y_reward_alg[::1000], label = 'DiSC-UCB', color = colors[1], linewidth=3)
plt.plot(x_value_r[::1000], y_baseline_alg[::1000], label = 'Conservative-Reward', linestyle = '--', color = colors[5], linewidth=3)

plt.ylim(0.0, 1.5)

plt.grid(True)
plt.xlabel('round,t', fontsize=25)
plt.ylabel('reward,r', fontsize=25)
plt.title('movielens data', fontsize=25)
plt.legend()
plt.show()
plt.close()